In [1]:
# initial setup
%run "../../../common/0_notebooks_base_setup.py"


/Users/csuarezgurruchaga/Desktop/Digital-House/CLASE_22/dsad_2021/common
default checking
Running command `conda list`... ok
jupyterlab=2.2.6 already installed
pandas=1.1.5 already installed
bokeh=2.2.3 already installed
seaborn=0.11.0 already installed
matplotlib=3.3.2 already installed
ipywidgets=7.5.1 already installed
pytest=6.2.1 already installed
chardet=4.0.0 already installed
psutil=5.7.2 already installed
scipy=1.5.2 already installed
statsmodels=0.12.1 already installed
scikit-learn=0.23.2 already installed
xlrd=2.0.1 already installed
Running command `conda install --yes nltk=3.5.0`... ok
Solving environment: ...working... done

# All requested packages already installed.


unidecode=1.1.1 already installed
pydotplus=2.0.2 already installed
pandas-datareader=0.9.0 already installed
flask=1.1.2 already installed


[<img src="https://www.digitalhouse.com/ar/logo-DH.png" width="400" height="200" align='right'>](http://digitalhouse.com.ar/)

# PRACTICA INDEPEDIENTE 

In [2]:
# Leemos los datos y seteamos el datetime como índice.
import numpy as np
import pandas as pd

bikes = pd.read_csv('../Data/bikes.csv', index_col='datetime', parse_dates=True)

In [3]:
# Dado que "count" es un método de pandas, es conviente renombrar la columna:

bikes.rename(columns={'count':'total'}, inplace=True)

##  Adicionales

### Ingeniería de features

Vea si puede crear las siguientes features:

- ** hora: ** como una sola feature numérica (de 0 a 23)
- ** hora: ** como una feature categórica (use 23 variables dummy)
- ** día: ** como una sola feature categórica (día = 1 de 7am a 8pm y día = 0 de lo contrario)


In [8]:
# hora como variable numérica
bikes['hora'] = bikes.index.hour

In [10]:
# hora como categorica

bikes_dummies = pd.get_dummies(bikes['hora'], drop_first=True)
bikes_dummies.sample(3)

,1,2,3,4,5,6,7,8,9,10,...,14,15,16,17,18,19,20,21,22,23
datetime,,,,,,,,,,,,,,,,,,,,,
2012-12-01 05:00:00,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2012-08-11 16:00:00,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
2011-11-14 18:00:00,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0


In [11]:
bikes_dummies.index = bikes.index

In [20]:
bikes = pd.concat([bikes,bikes_dummies], axis=1)

In [26]:
dia = (bikes.index.day >= 7) & (bikes.index.day<= 20)
bikes['dia'] = dia.astype(int)
bikes.sample(3)
bikes.columns

Index([    'season',    'holiday', 'workingday',    'weather',       'temp',
            'atemp',   'humidity',  'windspeed',     'casual', 'registered',
            'total',       'hora',        'dia',            1,            2,
                  3,            4,            5,            6,            7,
                  8,            9,           10,           11,           12,
                 13,           14,           15,           16,           17,
                 18,           19,           20,           21,           22,
                 23],
      dtype='object')

##### A continuación, pruebe utilizar cada una de las tres features con `train_test_rmse` para ver cuál funciona mejor!

In [23]:
from sklearn.linear_model import LinearRegression
from sklearn import metrics

In [24]:
# Definimos una función que acepta una lista de features, hace el split entre train y test,
# reservando un 25% de las observaciones para testeo, y devuelve la prueba RMSE.

from sklearn.model_selection import train_test_split

def train_test_rmse(feature_cols):
    X = bikes[feature_cols]
    y = bikes.total
    # Como estamos trabajando con observaciones ordenadas en el tiempo, ponemos
    # shuffle=False para evitar data leakage
    X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=False)
    linreg = LinearRegression()
    linreg.fit(X_train, y_train)
    y_pred = linreg.predict(X_test)
    return np.sqrt(metrics.mean_squared_error(y_test, y_pred))

In [29]:
feature_cols = ['temp', 'season', 'humidity','dia',1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23]
train_test_rmse(feature_cols)

153.9824579101825

### Crear y comparar modelos con variables cuadráticas

¿Con cuales variables probaría? 

In [32]:
bikes['temp2']=bikes.temp**2
bikes['humidity2'] = bikes.humidity ** 2

In [34]:
feature_cols = ['temp2', 'season', 'humidity2','dia',1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23]
train_test_rmse(feature_cols)

152.90714211686378